## ДЗ 1 (ОБЯЗАТЕЛЬНОЕ): Анализ температурных данных и мониторинг текущей температуры через OpenWeatherMap API

**Описание задания:**  
Вы аналитик в компании, занимающейся изучением климатических изменений и мониторингом температур в разных городах. Вам нужно провести анализ исторических данных о температуре для выявления сезонных закономерностей и аномалий. Также необходимо подключить API OpenWeatherMap для получения текущей температуры в выбранных городах и сравнить её с историческими данными.


### Цели задания:
1. Провести **анализ временных рядов**, включая:
   - Вычисление скользящего среднего и стандартного отклонения для сглаживания температурных колебаний.
   - Определение аномалий на основе отклонений температуры от $ \text{скользящее среднее} \pm 2\sigma $.
   - Построение долгосрочных трендов изменения температуры.
   - Любые дополнительные исследования будут вам в плюс.

2. Осуществить **мониторинг текущей температуры**:
   - Получить текущую температуру через OpenWeatherMap API.
   - Сравнить её с историческим нормальным диапазоном для текущего сезона.

3. Разработать **интерактивное приложение**:
   - Дать пользователю возможность выбрать город.
   - Отобразить результаты анализа температур, включая временные ряды, сезонные профили и аномалии.
   - Провести анализ текущей температуры в контексте исторических данных.


### Описание данных
Исторические данные о температуре содержатся в файле `temperature_data.csv`, включают:
  - `city`: Название города.
  - `timestamp`: Дата (с шагом в 1 день).
  - `temperature`: Среднесуточная температура (в °C).
  - `season`: Сезон года (зима, весна, лето, осень).

Код для генерации файла вы найдете ниже.

## Подготовка к выполнению работы

1. Импорт необходимых модулей

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests
import asyncio
import aiohttp
from multiprocessing import Pool, cpu_count
from datetime import datetime
import time
import nest_asyncio

nest_asyncio.apply()

2. Генерация необходимых данных

In [2]:
# Реальные средние температуры (примерные данные) для городов по сезонам
seasonal_temperatures = {
    "New York": {"winter": 0, "spring": 10, "summer": 25, "autumn": 15},
    "London": {"winter": 5, "spring": 11, "summer": 18, "autumn": 12},
    "Paris": {"winter": 4, "spring": 12, "summer": 20, "autumn": 13},
    "Tokyo": {"winter": 6, "spring": 15, "summer": 27, "autumn": 18},
    "Moscow": {"winter": -10, "spring": 5, "summer": 18, "autumn": 8},
    "Sydney": {"winter": 12, "spring": 18, "summer": 25, "autumn": 20},
    "Berlin": {"winter": 0, "spring": 10, "summer": 20, "autumn": 11},
    "Beijing": {"winter": -2, "spring": 13, "summer": 27, "autumn": 16},
    "Rio de Janeiro": {"winter": 20, "spring": 25, "summer": 30, "autumn": 25},
    "Dubai": {"winter": 20, "spring": 30, "summer": 40, "autumn": 30},
    "Los Angeles": {"winter": 15, "spring": 18, "summer": 25, "autumn": 20},
    "Singapore": {"winter": 27, "spring": 28, "summer": 28, "autumn": 27},
    "Mumbai": {"winter": 25, "spring": 30, "summer": 35, "autumn": 30},
    "Cairo": {"winter": 15, "spring": 25, "summer": 35, "autumn": 25},
    "Mexico City": {"winter": 12, "spring": 18, "summer": 20, "autumn": 15},
}

# Сопоставление месяцев с сезонами
month_to_season = {12: "winter", 1: "winter", 2: "winter",
                   3: "spring", 4: "spring", 5: "spring",
                   6: "summer", 7: "summer", 8: "summer",
                   9: "autumn", 10: "autumn", 11: "autumn"}

# Генерация данных о температуре
def generate_realistic_temperature_data(cities, num_years=10):
    dates = pd.date_range(start="2010-01-01", periods=365 * num_years, freq="D")
    data = []

    for city in cities:
        for date in dates:
            season = month_to_season[date.month]
            mean_temp = seasonal_temperatures[city][season]
            # Добавляем случайное отклонение
            temperature = np.random.normal(loc=mean_temp, scale=5)
            data.append({"city": city, "timestamp": date, "temperature": temperature})

    df = pd.DataFrame(data)
    df['season'] = df['timestamp'].dt.month.map(lambda x: month_to_season[x])
    return df

# Генерация данных
data = generate_realistic_temperature_data(list(seasonal_temperatures.keys()))
data.to_csv('temperature_data.csv', index=False)

## Ход работы

### **Этап 1. Анализ исторических данных**:
   - Вычислить **скользящее среднее** температуры с окном в 30 дней для сглаживания краткосрочных колебаний.
   - Рассчитать среднюю температуру и стандартное отклонение для каждого сезона в каждом городе.
   - Выявить аномалии, где температура выходит за пределы $ \text{среднее} \pm 2\sigma $.
   - Попробуйте распараллелить проведение этого анализа. Сравните скорость выполнения анализа с распараллеливанием и без него.

In [3]:
# Загрузка и небольшая предобработка типов колонок у исторических данных
data = pd.read_csv("temperature_data.csv")
data['timestamp'] = pd.to_datetime(data['timestamp'])

In [4]:
# Смотрим с чем работаем
data.head()

,city,timestamp,temperature,season
0,New York,2010-01-01,2.239064,winter
1,New York,2010-01-02,3.938989,winter
2,New York,2010-01-03,3.444667,winter
3,New York,2010-01-04,-6.047639,winter
4,New York,2010-01-05,-1.261620,winter


In [5]:
# Вычисление среднего и стандартного отклонения температуры по сезонам c помощью скользящего окна
def calculate_rolling_stats(city_data, window=30):
    city_data = city_data.copy()
    city_data['rolling_mean'] = city_data['temperature'].rolling(window=window, min_periods=1).mean()
    city_data['rolling_std'] = city_data['temperature'].rolling(window=window, min_periods=1).std()
    return city_data

data_windowed = calculate_rolling_stats(data)
data_windowed.head()

,city,timestamp,temperature,season,rolling_mean,rolling_std
0,New York,2010-01-01,2.239064,winter,2.239064,NaN
1,New York,2010-01-02,3.938989,winter,3.089026,1.202028
2,New York,2010-01-03,3.444667,winter,3.207573,0.874412
3,New York,2010-01-04,-6.047639,winter,0.893770,4.682357
4,New York,2010-01-05,-1.261620,winter,0.462692,4.168032


In [6]:
# Выявляем аномалии
def detect_anomalies(city_data):
    city_data['is_anomaly'] = (city_data['temperature'] < city_data['rolling_mean'] - 2 * city_data['rolling_std']) | \
                              (city_data['temperature'] > city_data['rolling_mean'] + 2 * city_data['rolling_std'])
    return city_data

data_with_anomalies = detect_anomalies(data_windowed)
data_with_anomalies[data_with_anomalies['is_anomaly'] == True]

,city,timestamp,temperature,season,rolling_mean,rolling_std,is_anomaly
24,New York,2010-01-25,-9.408704,winter,2.208171,4.499134,True
60,New York,2010-03-02,19.100432,spring,1.364080,5.799707,True
150,New York,2010-05-31,19.166038,spring,10.439582,4.138358,True
151,New York,2010-06-01,27.670042,summer,10.985076,5.199027,True
153,New York,2010-06-03,25.031008,summer,11.846482,5.916264,True
...,...,...,...,...,...,...,...
54622,Mexico City,2019-08-24,5.845879,summer,17.474273,5.153465,True
54658,Mexico City,2019-09-29,31.672787,autumn,15.874859,5.630954,True
54697,Mexico City,2019-11-07,26.598292,autumn,16.304451,5.031591,True
54715,Mexico City,2019-11-25,3.825891,autumn,15.922900,5.426966,True


#### Анализ распараллеливания пройденных процессов

In [7]:
# Объединим все функции, написанные выше, в одну (функция для одного города)
def analyze_city(city, city_data):
    city_data = city_data.copy()
    city_data = calculate_rolling_stats(city_data)
    city_data = detect_anomalies(city_data)

    avg_temp = city_data['temperature'].mean()
    min_temp = city_data['temperature'].min()
    max_temp = city_data['temperature'].max()
    return {
        "city": city,
        "average_temp": avg_temp,
        "min_temp": min_temp,
        "max_temp": max_temp,
        "anomalies": city_data[city_data['is_anomaly']]
    }

# Функция для всех городов (с возможностью распараллеливания)
def analyze_all_cities(data, parallel=True):
    cities = data['city'].unique()
    if parallel:
        num_processes = cpu_count()
        with Pool(num_processes) as pool:
            results = pool.starmap(analyze_city, [(city, data[data['city'] == city]) for city in cities])
    else:
        results = [analyze_city(city, data[data['city'] == city]) for city in cities]
    return results

In [8]:
# Для чистоты эксперимента прочитаем данные еще раз
data = pd.read_csv("temperature_data.csv")
data['timestamp'] = pd.to_datetime(data['timestamp'])

# Замер распараллеленного анализа
start_time = time.time()
results = analyze_all_cities(data, parallel=True)
end_time = time.time()
print('Parallel time taken: ', end_time - start_time, '\n')

# Замер последовательного анализа
start_time = time.time()
results = analyze_all_cities(data, parallel=False)
end_time = time.time()
print('\nSequental time taken: ', end_time - start_time, '\n')

Parallel time taken:  0.5155856609344482 


Sequental time taken:  0.37111425399780273 



**Причины, почему последовательное выполнение может быть быстрее (параллельное в данном случае также может выполнится быстрее, однако зачастую последовательное выполнение идет тут быстрее):**

1.	Накладные расходы на создание процессов - затрата времени на выделение ресурсов и запуск параллельных процессов приводит к тому, что при таком относительно небольшом объеме данных время, затрачиваемое на накладки с распараллеливанием процессов, превышает время последовательного исполнения процессов, у которого таких накладок нет.

2.	Передача данных между процессами - для обработки в параллельных процессах данные делятся на части и передаются через межпроцессное взаимодействие, что также добавляет задержки.

> В последовательном режиме отсутствуют накладки на создание параллельных процессов или передачу в них разбиенных данных, что делает его быстрее при обработке малых объемов данных.

## **Этап 2. Мониторинг текущей температуры**
   - Подключить OpenWeatherMap API для получения текущей температуры города. Для получения API Key (бесплатно) надо зарегистрироваться на сайте. Обратите внимание, что API Key может активироваться только через 2-3 часа, это нормально. Посему получите ключ заранее.
   - Получить текущую температуру для выбранного города через OpenWeatherMap API.
   - Определить, является ли текущая температура нормальной, исходя из исторических данных для текущего сезона.
   - Данные на самом деле не совсем реальные (сюрпрайз). Поэтому на момент эксперимента погода в Берлине, Каире и Дубае была в рамках нормы, а в Пекине и Москве аномальная. Протестируйте свое решение для разных городов.
   - Попробуйте для получения текущей температуры использовать синхронные и асинхронные методы. Что здесь лучше использовать?

In [9]:
# API настройки
API_KEY = "a9b6800fa5d4032abeb9ca6b062cd037"
BASE_URL = "http://api.openweathermap.org/data/2.5/weather"

# Загрузка данных (опять же, для чистоты эксперимента)
data = pd.read_csv("temperature_data.csv")
data["timestamp"] = pd.to_datetime(data["timestamp"])

# Зададим список интересующих нас городов из датасета, который мы генерили
cities = data["city"].unique()

# Текущий период
current_month = datetime.now().month
season_map = {12: "winter", 1: "winter", 2: "winter",
              3: "spring", 4: "spring", 5: "spring",
              6: "summer", 7: "summer", 8: "summer",
              9: "autumn", 10: "autumn", 11: "autumn"}
current_season = season_map[current_month]

# Сбор сезонных статистик
season_stats = data[data["season"] == current_season].groupby("city")["temperature"].agg(["mean", "std"]).reset_index()

#### Блок с используемыми функциями

In [10]:
def fetch_weather_sync(city):
    """
    Синхронное получение текущей температуры для города
    """
    try:
        response = requests.get(
            BASE_URL,
            params={"q": city, "appid": API_KEY, "units": "metric"}
        )
        response.raise_for_status()
        data = response.json()
        return city, data["main"]["temp"]
    except requests.RequestException as e:
        print(f"Ошибка для города {city}: {e}")
        return city, None


async def fetch_weather_async(city, session):
    """
    Асинхронное получение текущей температуры для города
    """
    try:
        async with session.get(
            BASE_URL,
            params={"q": city, "appid": API_KEY, "units": "metric"}
        ) as response:
            data = await response.json()
            return city, data["main"]["temp"]
    except Exception as e:
        print(f"Ошибка для города {city}: {e}")
        return city, None


async def fetch_all_weather_async(cities):
    """
    Асинхронный сбор текущей температуры для всех городов
    """
    async with aiohttp.ClientSession() as session:
        tasks = [fetch_weather_async(city, session) for city in cities]
        results = await asyncio.gather(*tasks)
    return dict(results)


def get_temperatures_sync(cities):
    """
    Синхронное получение текущих температур для всех городов
    """
    return {city: fetch_weather_sync(city)[1] for city in cities}


async def get_temperatures_async(cities):
    """
    Асинхронное получение текущих температур для всех городов
    """
    return await fetch_all_weather_async(cities)


def check_temperature_anomaly(city, current_temp, season_stats):
    """
    Проверяет, является ли температура аномальной
    """
    city_stats = season_stats[season_stats["city"] == city]
    if city_stats.empty:
        return False

    mean_temp = city_stats["mean"].values[0]
    std_temp = city_stats["std"].values[0]
    lower_bound = mean_temp - 2 * std_temp
    upper_bound = mean_temp + 2 * std_temp

    return lower_bound <= current_temp <= upper_bound


#### Обращение к API

1. Обращаемся за актуальной температурой разных городов в момент запроса в синхронном и асинхронном формате
2. Анализируем аномальность текущих температур разных городов на основе

In [11]:
# Делаем синхронный запрос

start_time = time.time()
current_temperatures_sync = get_temperatures_sync(cities)
end_time = time.time()
print(f"Синхронный запрос выполнен за {end_time - start_time:.2f} секунд.")

Синхронный запрос выполнен за 0.98 секунд.


In [12]:
# Делаем асинхронный запрос

request_time_async = datetime.now()
start_time = time.time()
current_temperatures_async = asyncio.run(get_temperatures_async(cities))
end_time = time.time()
print(f"Асинхронный запрос выполнен за {end_time - start_time:.2f} секунд.")

Асинхронный запрос выполнен за 0.12 секунд.


> Как мы видим, асинхронные методы лучше, потому что они выполняют запросы параллельно, экономят время и подходят для работы с бОльшим количеством городов. Синхронный подход проще, но медленнее, особенно при множественных запросах. Посмотрим результаты на примере сделанного асинхронного запроса:

In [13]:
# Результат асинхронного запроса: вывод текущих температур
print(f"\nТекущие температуры в городах в момент {request_time_async}:")
for city, temp in current_temperatures_async.items():
    if temp is not None:
        print(f"{city}: {temp:.2f}°C")
    else:
        print(f"{city}: ошибка получения данных")


Текущие температуры в городах в момент 2025-01-05 23:50:31.019714:
New York: -0.73°C
London: 12.15°C
Paris: 11.74°C
Tokyo: 3.81°C
Moscow: -3.73°C
Sydney: 30.79°C
Berlin: 3.82°C
Beijing: -1.06°C
Rio de Janeiro: 25.75°C
Dubai: 16.96°C
Los Angeles: 20.91°C
Singapore: 25.68°C
Mumbai: 21.99°C
Cairo: 15.42°C
Mexico City: 18.86°C


In [14]:
# Результат асинхронного запроса: проверка на аномальность и вывод результатов
results = {}
for city, current_temp in current_temperatures_async.items():
    if current_temp is not None:
        results[city] = check_temperature_anomaly(city, current_temp, season_stats)

for city, is_normal in results.items():
    print(f"{city}: {'Норма' if is_normal else 'Аномалия'}")

New York: Норма
London: Норма
Paris: Норма
Tokyo: Норма
Moscow: Норма
Sydney: Аномалия
Berlin: Норма
Beijing: Норма
Rio de Janeiro: Норма
Dubai: Норма
Los Angeles: Норма
Singapore: Норма
Mumbai: Норма
Cairo: Норма
Mexico City: Норма
